In [1]:
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")
library("DESeq2")
getwd()

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain intro

[1] "/home/gstupp/projects/Wolan/ana_extractions/H1"

In [16]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable$technical=factor(sampleTable$technical)
sampleTable$sample_type=factor(sampleTable$sample_type)
sampleTable$sample_type <- relevel(sampleTable$sample_type, ref="whole")
sampleTable

,technical,sample_type,sample_name,path
H1_PelletTriflic_Fl_R1,1,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R1.txt
H1_PelletTriflic_Fl_R2,2,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R2.txt
H1_PelletTriflic_Fl_R3,3,sonic_pellet,H1_PelletTriflic_Fl,data/H1_PelletTriflic_FlFusion_5ppm_10ppmfrag/H1_PelletTriflic_Fl_R3.txt
H1_SonicSupTriflic_V2_R1,1,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R1.txt
H1_SonicSupTriflic_V2_R2,2,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R2.txt
H1_SonicSupTriflic_V2_R3,3,sonic_sup,H1_SonicSupTriflic_V2,data/H1_SonicSupTriflic_FlFusion_5ppm_10ppmfrag/H1_SonicSupTriflic_V2_R3.txt
H1_wholetriflic_R1,1,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R1.txt
H1_wholetriflic_R2,2,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R2.txt
H1_wholetriflic_R3,3,whole,H1_wholetriflic,data/H1_WholeTriflic_FlFusion_5ppm_10ppmfrag/H1_wholetriflic_R3.txt


In [17]:
countData = read.csv("out/df_filt.csv", row.names = 1)
colnames(countData) = lapply(colnames(countData), function(x) {gsub("\\.", "_", x)})
locusTable = read.csv("out/locus_df.csv", row.names = 1)
sampleTable = sampleTable[colnames(countData),]

In [18]:
dds = DESeqDataSetFromMatrix(countData, colData = sampleTable, ~ sample_type)

## collapse replicates
dds = collapseReplicates(dds, dds$sample_name)
sampleTable = colData(dds)

In [20]:
head(counts(dds))

,H1_PelletTriflic_Fl,H1_SonicSupTriflic_V2,H1_wholetriflic
202956,22,30,34
209005,99,49,90
217964,7,0,17
248190,7,0,13
251275,13,0,12
265827,23,0,1


In [21]:
dds = DESeq(dds)

estimating size factors
estimating dispersions
Warning message in checkForExperimentalReplicates(object, modelMatrix):
“same number of samples and coefficients to fit,
  estimating dispersion by treating samples as replicates.
  please read the ?DESeq section on 'Experiments without replicates'.
  in summary: this analysis only potentially useful for data exploration,
  accurate differential expression analysis requires replication”gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [22]:
#mcols(dds)<-cbind(mcols(dds),locusTable)
save(dds, file = "out/dds")

In [31]:
res = results(dds, contrast=c("sample_type","sonic_pellet","whole"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 0, 0% 
LFC < 0 (down)   : 0, 0% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [32]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_pellet_v_whole_deseq_results.csv")

In [33]:
res = results(dds, contrast=c("sample_type","sonic_sup","whole"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 6, 0.074% 
LFC < 0 (down)   : 63, 0.78% 
outliers [1]     : 0, 0% 
low counts [2]   : 7521, 93% 
(mean count < 78)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [34]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_sup_v_whole_deseq_results.csv")

In [35]:
res = results(dds, contrast=c("sample_type","sonic_sup","sonic_pellet"))
summary(res)


out of 8083 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2, 0.025% 
LFC < 0 (down)   : 67, 0.83% 
outliers [1]     : 0, 0% 
low counts [2]   : 7521, 93% 
(mean count < 78)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [36]:
res = cbind(res, locusTable)
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/sonic_sup_v_pellet_deseq_results.csv")